**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *
from scripts.postprocessing import *
from scripts.protein_preparation import *

[12:39:48] Initializing Normalizer
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/sklearn_extra/cluster/_commonnn.py:18: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(sklearn.__version__) < LooseVersion("0.23.0"):
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: 

In [2]:
software = Path('/home/tony/DockM8/software')
receptor = Path('/home/tony/DockM8/testing_single_docking/protein.pdb')
ref_file = Path('/home/tony/DockM8/testing_single_docking/ref.sdf')
docking_library = Path('/home/tony/DockM8/testing_single_docking/library.sdf')
docking_programs = ['GNINA', 'SMINA', 'PLANTS', 'QVINAW', 'QVINA2']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS', 'bestpose_QVINAW', 'bestpose_QVINA2']
rescoring_functions = ['GNINA_Affinity','CNN-Score','CNN-Affinity', 'AD4', 'CHEMPLP', 'RFScoreVS', 'LinF9', 'SCORCH', 'Vinardo', 'PLECScore', 'NNScore', 'KORPL', 'ConvexPLR', 'AAScore']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'GypsumDL'
ncpus = 5
pocket = 'reference'
prepare_protein = 1

In [3]:
if prepare_protein == 1:
    #Prepare the protein
    prepared_receptor = prepare_protein_protoss(receptor)
else:
    prepared_receptor = receptor

#Create a temporary folder for all further calculations
w_dir = prepared_receptor.parent / prepared_receptor.stem
print('The working directory has been set to:', w_dir)
(w_dir).mkdir(exist_ok=True)

if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, prepared_receptor, 8)
    print(pocket_definition)
if pocket == 'RoG':
    pocket_definition = get_pocket_RoG(ref_file, prepared_receptor)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(prepared_receptor, w_dir, method='volume')
    print(pocket_definition)


[2023-Nov-18 12:39:52]: Preparing protein with ProtoSS ...
Job 2f7717e1-2782-41dc-af34-a32d7a9f1099 completed with success
The working directory has been set to: /home/tony/DockM8/testing_single_docking/protein_protoss

[2023-Nov-18 12:39:54]: Extracting pocket from /home/tony/DockM8/testing_single_docking/protein_protoss.pdb using /home/tony/DockM8/testing_single_docking/ref.sdf as reference ligand

[2023-Nov-18 12:40:15]: Finished extracting pocket from /home/tony/DockM8/testing_single_docking/protein_protoss.pdb using /home/tony/DockM8/testing_single_docking/ref.sdf as reference ligand
{'center': [16.7, -2.69, 17.61], 'size': [16.0, 16.0, 16.0]}


In [ ]:
if (w_dir / 'final_library.sdf').is_file() == False:
    prepare_library(docking_library, w_dir, id_column, protonation, software, ncpus)

In [ ]:
docking(w_dir, prepared_receptor, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5, 'multiprocessing')
concat_all_poses(w_dir, docking_programs, prepared_receptor)


In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(str(w_dir / 'allposes.sdf'), idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
print(f'Loaded {len(all_poses)} poses.')
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

for metric in clustering_metrics:
        if os.path.isfile(w_dir / 'clustering/'f'{metric}_clustered.sdf') == False:
            cluster_pebble(metric, 'KMedoids', w_dir, prepared_receptor, all_poses, ncpus)

In [ ]:
for metric in clustering_metrics:
        rescore_all(w_dir, prepared_receptor, pocket_definition, software, w_dir / 'clustering' / f'{metric}_clustered.sdf', rescoring_functions , ncpus)


In [6]:
apply_consensus_methods(w_dir, 'bestpose', 'method6', rescoring_functions, 'min_max')